In [1]:
import flash
import mlflow
import torch
from flash.core.data.utils import download_data
from flash.text import TextClassificationData, TextClassifier
import torchmetrics
from lightning.pytorch.callbacks import ModelSummary
#from pytorch_lightning.core.memory import ModelSummary
import shutil
import tempfile
import os


/Users/mingzhang/miniconda3/envs/pdl/lib/python3.8/site-packages/flash/__init__.py:21: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/Users/mingzhang/miniconda3/envs/pdl/lib/python3.8/site-packages/flash/__init__.py:21: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):


In [2]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://100.65.79.57:9000"

In [3]:
def save_file_as_mlflow_artifact(file_content_string, file_name = 'model_summary.txt'):
    tempdir = tempfile.mkdtemp()
    try:
        summary_file = os.path.join(tempdir, file_name)
        with open(summary_file, "w") as f:
            f.write(file_content_string)

        mlflow.log_artifact(local_path=summary_file)
    finally:
        shutil.rmtree(tempdir)

In [4]:
download_data("https://pl-flash-data.s3.amazonaws.com/imdb.zip", "./data/")
datamodule = TextClassificationData.from_csv(
    "review",
    "sentiment",
    train_file="data/imdb/train.csv",
    val_file="data/imdb/valid.csv",
    test_file="data/imdb/test.csv",
    batch_size=4,
)

Map:   0%|          | 0/22500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/Users/mingzhang/miniconda3/envs/pdl/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
# if we want to track multiple metrics simultaneously, define a list of metrics and pass it to the classifier metric parameter
list_of_metrics = [torchmetrics.Accuracy(task="multiclass", num_classes=datamodule.num_classes, top_k=1),
                    torchmetrics.F1Score(task="multiclass", num_classes=datamodule.num_classes, top_k=1),
                    torchmetrics.Precision(task="multiclass", num_classes=datamodule.num_classes, top_k=1),
                    torchmetrics.Recall(task="multiclass", num_classes=datamodule.num_classes, top_k=1)]

In [13]:
# uncomment the following line and comment out the immediate next line to use a list of metrics instead of a single metric during model training
# classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes, metrics=list_of_metrics)

classifier_model = TextClassifier(backbone="prajjwal1/bert-tiny", num_classes=datamodule.num_classes, metrics=torchmetrics.F1Score(task="multiclass", num_classes=datamodule.num_classes, top_k=1))
trainer = flash.Trainer(max_epochs=1, accelerator="mps", devices=1)

/Users/mingzhang/miniconda3/envs/pdl/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:269: Attribute 'metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['metrics'])`.
Using 'prajjwal1/bert-tiny' provided by Hugging Face/transformers (https://github.com/huggingface/transformers).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [14]:
EXPERIMENT_NAME = "dl_model_chapter03"
mlflow.set_tracking_uri('http://100.65.79.57')
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 1


In [15]:
MODEL_ARTIFACT_PATH = 'dl_model'
REGISTERED_MODEL_NAME = 'nlp_dl_model'
with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="chapter03") as dl_model_tracking_run:
    trainer.finetune(classifier_model, datamodule=datamodule, strategy="freeze")
    #trainer.test()

    # mlflow log metrics
    cur_metrics = trainer.callback_metrics
    # Cast metric value as  float before passing into logger.
    metrics = dict(map(lambda x: (x[0], float(x[1])), cur_metrics.items()))
    mlflow.log_metrics(metrics)

    # mlflow log parameters
    params = {"epochs": trainer.max_epochs}
    if hasattr(trainer, "optimizers"):
        optimizer = trainer.optimizers[0]
        params["optimizer_name"] = optimizer.__class__.__name__
    if hasattr(optimizer, "defaults"):
        params.update(optimizer.defaults)
    # add hyper-parameters used in this training
    params.update(classifier_model.hparams)
    mlflow.log_params(params)

    # log model summary as an artifact
    summary = ModelSummary(classifier_model)
    save_file_as_mlflow_artifact(str(summary))

    # log and register the trained model
    mlflow.pytorch.log_model(pytorch_model=classifier_model, artifact_path=MODEL_ARTIFACT_PATH, registered_model_name=REGISTERED_MODEL_NAME)



  | Name          | Type               | Params
-----------------------------------------------------
0 | train_metrics | ModuleDict         | 0     
1 | val_metrics   | ModuleDict         | 0     
2 | test_metrics  | ModuleDict         | 0     
3 | adapter       | HuggingFaceAdapter | 4.4 M 
-----------------------------------------------------
258       Trainable params
4.4 M     Non-trainable params
4.4 M     Total params
17.545    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
Registered model 'nlp_dl_model' already exists. Creating a new version of this model...
2024/03/10 15:47:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nlp_dl_model, version 4
Created version '4' of model 'nlp_dl_model'.


In [16]:
run_id = dl_model_tracking_run.info.run_id
print("run_id: {}; lifecycle_stage: {}".format(run_id,
    mlflow.get_run(run_id).info.lifecycle_stage))

run_id: 2c33d1f109d1467ab81ded8374e4a87b; lifecycle_stage: active


In [17]:
logged_model = f'runs:/{run_id}/{MODEL_ARTIFACT_PATH}'
# Load model as a pytorch model, not as the pyfunc model
model = mlflow.pytorch.load_model(logged_model)
# To bypass a lightning-flash's bug, we need to set the stage to test so a loaded model can be used to do prediction
model.trainer.state.stage='test'
trainer = flash.Trainer(max_epochs=1, accelerator="mps", devices=1)
datamodule = TextClassificationData.from_lists(
    predict_data=[
        "Turgid dialogue, feeble characterization - Harvey Keitel a judge?.",
        "The worst movie in the history of cinema.",
        "This guy has done a great job with this movie!",
    ],
    batch_size=4,
)
predictions = trainer.predict(model, datamodule=datamodule, output="labels")
print(predictions)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

[[0, 1, 0]]
